In [ ]:
import requests
import json
import csv
from datetime import datetime, timezone
from time import sleep

#### 輸出的CSV
# 輸入地點
location = "饒河夜市"
output_file = f'reviews_output_{location}.csv'

#### ID不重複
unique_ids = set()

#### 計算評論數 & 有內文的評論數
total_reviews = 0
reviews_with_comments = 0

#### 寫入CSV標題 (不建議使用 若遇到接續爬的時候 忘了註解 會洗掉前面紀錄)
# with open(output_file, mode='w', encoding='utf-8', newline='') as file:
#     writer = csv.writer(file)
#     writer.writerow(["ID", "Rating", "TIME", "Comment"])

# 紀錄2s碼的串列
list_2s = []

# 清空串列
list_2s.clear()

# 計數器
count = 1

# 手動取得2s 碼 (評論排序選最新 去F12 console 的 Networking 觀察評論載入時 有個listugcposts的項目 網址中2s 後方的項目為2s碼 網址其他部分貼到下方)
google = "CAESYENBRVFDaHBDUTJrNFNVRlNTVzVEWjI5QlVEY3lSM1J1T1ZaV1psOWZSV2hCWWt4NGNFSk1NV3BXVjNZM1pYTkRUVUZCUVVGQlIyZG1YMEZ0ZEc0NVZsWm1SMEZCYVVGQg%3D%3D"

#### 發出請求
# 抓到沒有下一個 2s 碼為止
while type(google)==str:
    # 紀錄每一頁的 2s 碼 若意外中斷可以印出中斷的2s碼接續爬
    list_2s.append(
        {
        'page': count,
        'code_2s': google
    }
    )
    
    # 手動取得網址 方法寫在上方
    url = f'https://www.google.com/maps/rpc/listugcposts?authuser=0&hl=zh-TW&gl=tw&pb=!1m6!1s0x3442ab9c0db4a583%3A0x3da21183815df6f6!6m4!4m1!1e1!4m1!1e3!2m2!1i10!2s{google}!5m2!1s60SUZ7iVL7rT2roPgpr0-QU!7e81!8m9!2b1!3b1!5b1!7b1!12m4!1b1!2b1!4m1!1e1!11m0!13m1!1e2'
    response = requests.get(url)
    response.raise_for_status()

    raw_content = response.text.strip()

    #### 移除開頭的)]}'
    if raw_content.startswith(")]}'"):
        raw_content = raw_content[4:]

    data = json.loads(raw_content)

    google = data[1].replace('=', '%3D')

    sleep(1)
    #### 提取要的資料
    with open(output_file, mode='a', encoding='utf-8', newline='') as file:
        writer = csv.writer(file)
        for i in range(10):
            try:                        
                review_id =  str(data[2][i][0][0])
                rating = str(data[2][i][0][2][0][0])
                timestamp_seconds = data[2][i][0][1][2] // 1000000
                formatted_date = datetime.fromtimestamp(timestamp_seconds, tz=timezone.utc).strftime('%Y-%m-%d')
                try:
                    comment = data[2][i][0][2][15][0][0]
                except (IndexError, TypeError):
                    comment = ""
                # 確保 ID 唯一，不重複儲存
                if review_id not in unique_ids:
                    writer.writerow([review_id, rating, formatted_date, comment])
                    unique_ids.add(review_id)
                    total_reviews += 1  # 增加計數
                    if comment.strip():  # 若評論內容不為空
                            reviews_with_comments += 1
            except Exception as e:
                print(f"評論 {i + 1}: 無評論或格式錯誤 ({e})")
    print(f"現在在第 {count} 頁")
    print(f"總共存入 {total_reviews} 筆評論")
    print(f"其中有 {reviews_with_comments} 筆評論有內文")
    count += 1
print("沒有下一頁")

In [ ]:
# 取得所有爬過的 2s 碼
print(list_2s)

# 若意外中斷可以取得該頁 2s 碼接續
print(google)